# 数据准备用于微调


在本教程中，我们将展示微调的第一个步骤：数据集准备的示例。



## 0. 安装

In [ ]:
%pip install -U datasets

# 微调准备

假设我们希望对金融任务进行模型微调。我们找到了一个可能有用的开源数据集：[financial-qa-10k](https://huggingface.co/datasets/virattt/financial-qa-10K)。让我们看看如何正确准备数据集用于微调。

原始数据集具有以下结构：
- 5个列：'question'（问题），'answer'（回答），'context'（上下文），'ticker'（股票代码），和'filing'（申报文件）。
- 7000行数据。

In [4]:
from datasets import load_dataset

ds = load_dataset("virattt/financial-qa-10K", split="train")
ds

# 数据格式说明：
# - question: 金融相关问题，例如关于公司财务状况、业务运营等的问询
# - answer: 对问题的回答，通常摘自公司的财务报告
# - context: 问题的背景信息，通常是从财务文件中提取的原始文本段落
# - ticker: 股票代码，标识相关公司的股票市场符号
# - filing: 财务申报文件信息，如10-K（年度报告）或10-Q（季度报告）的引用

c:\Users\k\.conda\envs\XiaokeAILabs\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\k\.conda\envs\XiaokeAILabs\lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\k\.cache\huggingface\hub\datasets--virattt--financial-qa-10K. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate dev

Dataset({
    features: ['question', 'answer', 'context', 'ticker', 'filing'],
    num_rows: 7000
})

## 1. 微调用数据

构建符合以下格式的数据集：

``` python
{"query": str, "pos": List[str], "neg":List[str], "pos_scores": List[int], "neg_scores": List[int], "prompt": str, "type": str}
```

`query` 是查询语句，`pos` 是一个正向文本列表，`neg` 是一个负向文本列表。`pos_scores` 是对应查询和正向文本的分数列表，`neg_scores` 是对应查询和负向文本的分数列表，如果你不使用知识蒸馏，可以忽略这两项。`prompt` 是用于查询的提示语，它会覆盖查询检索指令。`type` 用于 bge-en-icl，包括 `normal`、`symmetric_class`、`symmetric_clustering` 等类型。如果查询没有负向文本，你可以从整个语料库中随机抽样一些作为负向样本。

我们选择 'question' 和 'context' 列作为我们的查询和回答（正向样本），并重命名这些列。然后添加 'id' 列用于后续评估。

In [6]:
ds = ds.select_columns(column_names=["question", "context"])
ds = ds.rename_column("question", "query")
ds = ds.rename_column("context", "pos")
ds = ds.add_column("id", [str(i) for i in range(len(ds))])
ds[0]

# {
#   'query': '英伟达（NVIDIA）在扩展到其他计算密集型领域之前，最初专注于哪个领域？',
#   'pos': '自从我们最初专注于个人电脑图形处理以来，我们已经扩展到其他几个大型且重要的计算密集型领域。',
#   'id': '0'
# }

{'query': 'What area did NVIDIA initially focus on before expanding to other computationally intensive fields?',
 'pos': 'Since our original focus on PC graphics, we have expanded to several other large and important computationally intensive fields.',
 'id': '0'}

负向样本在嵌入模型训练中非常重要。我们的初始数据集没有负向文本，因此我们直接从整个语料库中抽取一些样本。

In [7]:
import numpy as np

# 设置每个查询对应的负样本数量
neg_num = 10

def str_to_lst(data):
    # 将单个字符串转换为列表，使格式符合微调要求
    # 微调数据格式要求'pos'字段为列表形式
    data["pos"] = [data["pos"]]
    return data

# 为每个查询采样负例文本
# 注意这里是使用随机采样的方式来生成负样本，实际场景里面我们应该使用大模型或者其他方式获取真正的负样本。
new_col = []
for i in range(len(ds)):
    # 从数据集中随机采样neg_num个索引作为负样本
    ids = np.random.randint(0, len(ds), size=neg_num)
    # 确保不会将当前样本自身作为负样本
    while i in ids:
        ids = np.random.randint(0, len(ds), size=neg_num)
    # 根据采样的索引获取对应的文本作为负样本
    neg = [ds[i.item()]["pos"] for i in ids]
    new_col.append(neg)
# 将采样得到的负样本添加到数据集中
ds = ds.add_column("neg", new_col)

# 将'pos'键的值转换为列表格式
# 通过map函数对数据集中的每一行应用str_to_lst函数
ds = ds.map(str_to_lst)

Map: 100%|██████████| 7000/7000 [00:00<00:00, 13704.86 examples/s]


最后，我们添加用于查询的提示语。在推理过程中，它将作为 `query_instruction_for_retrieval`。

In [8]:
instruction = "Represent this sentence for searching relevant passages: "
ds = ds.add_column("prompt", [instruction]*len(ds))

现在数据集的单行样例如下：

In [9]:
ds[0]

{'query': 'What area did NVIDIA initially focus on before expanding to other computationally intensive fields?',
 'pos': ['Since our original focus on PC graphics, we have expanded to several other large and important computationally intensive fields.'],
 'id': '0',
 'neg': ['Kroger expects that its value creation model will deliver total shareholder return within a target range of 8% to 11% over time.',
  'CSB purchased First Mortgages of $2.9 billion during 2023.',
  'See Note 13 to our Consolidated Financial Statements for information on certain legal proceedings for which there are contingencies.',
  'Diluted earnings per share were $16.69 in fiscal 2022 compared to $15.53 in fiscal 2021.',
  'In the year ended December 31, 2023, Total net sales and revenue increased primarily due to: (1) increased net wholesale volumes primarily due to increased sales of crossover vehicles and full-size pickup trucks, partially offset by decreased sales of mid-size pickup trucks; (2) favorable Pri

然后我们将数据集分割为训练集和测试集。

In [17]:
# 将数据集分割为训练集和测试集
# test_size=0.1 表示将10%的数据用于测试集
# shuffle=True 表示在分割前对数据进行随机打乱
# seed=520 设置随机种子，确保每次运行结果一致
split = ds.train_test_split(test_size=0.1, shuffle=True, seed=520)

# 从分割结果中获取训练集和测试集
train = split["train"]  # 包含90%的数据，用于模型训练
test = split["test"]    # 包含10%的数据，用于模型评估

# 至此，我们已经成功将原始数据集分为两部分：
# 1. train - 用于微调模型的训练数据
# 2. test - 用于评估微调后模型性能的测试数据

现在我们可以存储数据以供后续微调使用：

In [11]:
train.to_json("ft_data/training.json")

Creating json from Arrow format: 100%|██████████| 7/7 [00:00<00:00, 28.16ba/s]


16583481

## 2. 用于评估的测试数据

最后一步是构建用于评估的测试数据集。

In [12]:
test

Dataset({
    features: ['query', 'pos', 'id', 'neg', 'prompt'],
    num_rows: 700
})

首先选择查询所需的列：

In [13]:
# 从测试集中选择 ID 和查询列，用于后续评估
queries = test.select_columns(column_names=["id", "query"])
# 将原 "query" 列重命名为 "text"，这样以便后续处理
queries = queries.rename_column("query", "text")
# 显示第一条数据，验证数据格式是否正确
queries[0]

# 这段代码做了以下几件事情：
# 1. 从测试数据集(test)中选择两列："id"和"query"
#   - id：每个查询的唯一标识符
#   - query：查询文本内容，即用户的问题
# 2. 将"query"列重命名为"text"，符合评估时的数据格式要求
#   - 这是因为大多数评估框架期望查询数据使用"text"字段名
# 3. 输出处理后数据集的第一条记录，用于验证数据格式转换是否正确
#
# 该处理后的数据集将用于微调后的模型评估，评估模型针对这些查询返回
# 相关文档的能力。处理后每条数据包含唯一ID和对应的查询文本。

{'id': '1289',
 'text': 'How does Starbucks recognize the interest and penalties related to income tax matters on their financial statements?'}

然后选择语料库所需的列：

In [ ]:
# 为语料库选择必要的列：id（唯一标识符）和pos（文本内容）
corpus = ds.select_columns(column_names=["id", "pos"])

# 将"pos"列重命名为"text"，符合后续评估框架的标准格式
# 大多数评估框架期望语料库中的文本字段使用"text"作为字段名
corpus = corpus.rename_column("pos", "text")

# 至此，语料库数据集已包含两个关键列：
# 1. id: 每个文档的唯一标识符，用于在评估时关联查询和相关文档
# 2. text: 文档的实际内容，即财务报告中的文本段落
#
# 这个语料库将用于评估微调后的模型，模型需要从这个语料库中
# 检索与用户查询最相关的文档。正确的检索表明模型能够理解
# 金融文本的语义并建立查询与文档间的关联。

最后，创建指示查询与相应语料库关系的 qrels：

In [15]:
# 创建qrels（查询与相关文档对应关系）
# qrels是用于评估检索系统性能的标准格式，记录了每个查询与相关文档的关系
# 首先从test数据集中选择id列，这将作为查询ID
qrels = test.select_columns(["id"])

# 将id列重命名为qid（query id），遵循评估格式标准
qrels = qrels.rename_column("id", "qid")

# 添加docid列，表示与每个查询相关的文档ID
# 在这个例子中，我们直接使用测试集中的ID作为相关文档ID
# 这意味着对于每个查询，我们认为测试集中的对应文档是相关的
qrels = qrels.add_column("docid", list(test["id"]))

# 添加relevance（相关性）列，表示查询和文档之间的相关程度
# 这里所有关系都标记为1，表示所有指定的文档对于对应查询都是相关的
# 在更复杂的评估中，相关性可能有不同级别（如0-3）表示不相关到高度相关
qrels = qrels.add_column("relevance", [1]*len(test))

# 显示第一条记录，验证数据格式是否正确
qrels[0]

# 至此，我们已经构建了用于评估的qrels数据集，包含三个关键列：
# 1. qid: 查询的唯一标识符
# 2. docid: 与查询相关的文档标识符
# 3. relevance: 查询与文档的相关性得分（这里统一为1）
# 
# 这个qrels数据集将用于评估模型的检索性能，评估时会比较模型返回的文档
# 与这个qrels中记录的相关文档，计算各种评估指标如MAP、MRR、NDCG等

Flattening the indices: 100%|██████████| 700/700 [00:00<00:00, 31334.18 examples/s]


{'qid': '1289', 'docid': '1289', 'relevance': 1}

存储数据集

In [16]:
queries.to_json("ft_data/test_queries.jsonl")
corpus.to_json("ft_data/corpus.jsonl")
qrels.to_json("ft_data/test_qrels.jsonl")

Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 501.59ba/s]


30574